In [2]:
import xml.etree.cElementTree as et
import urllib3
#import numpy as np
import pandas as pd
import re
import sqlite3

db_con = sqlite3.connect("mydatabase.db")
cursor = db_con.cursor()

#cursor.executescript('drop table if exists corpus;')
cursor.execute("""CREATE TABLE if not exists corpus
                  (name text, party text, function text, maintext text) 
               """)

http = urllib3.PoolManager()

# API key is curently not needed, but may be in the future?
# may register at: http://data.stortinget.no/no/dokumentasjon-og-hjelp/api-nokkel/
# if not valid key or empty, gives 'bad request' error
API_key = ""

In [ ]:
# first fetch ids of all sessions
sesjoner_url = 'http://data.stortinget.no/eksport/sesjoner?key=' + API_key
sesjoner_respons = http.request('GET', sesjoner_url)
sesjoner_root = et.fromstring(sesjoner_respons.data)

sesjoner_liste = sesjoner_root.find('{http://data.stortinget.no}sesjoner_liste')
sesjoner = []

for sesjon in sesjoner_liste.findall('{http://data.stortinget.no}sesjon'):
    sesjoner.append(sesjon.find('{http://data.stortinget.no}id').text)

print('Got ' + str(len(sesjoner)) + ' sessions.')

# hardcode list of all sessions, based on the above on 2018-03-25, 
#sesjoner = ['2018-2019',  '2017-2018', '2016-2017', '2015-2016', '2014-2015', '2013-2014', '2012-2013', 
#         '2011-2012', '2010-2011', '2009-2010', '2008-2009', '2007-2008', '2006-2007', '2005-2006', 
#         '2004-2005', '2003-2004', '2002-2003', '2001-2002', '2000-2001', '1999-2000', '1998-99', 
#         '1997-98', '1996-97', '1995-96', '1994-95', '1993-94', '1992-93', '1991-92', '1990-91', 
#         '1989-90', '1988-89', '1987-88', '1986-87']

# publications from http://data.stortinget.no/eksport/ got a new format starting from the 2016-2017, session, therefore save the index off that session:
nytt_format_ix = sesjoner.index('2016-2017')

In [ ]:
# next fetch all publications
# create tw olists to deal with format change
publikasjoner = []
publikasjoner_eldreformat = []

iterasjon = 0
for sesjonsid in sesjoner:
    print('\rImport iterasjon ' + str(iterasjon + 1) + ' of ' + str(len(sesjoner)),  end='')
    publikasjoner_url = 'http://data.stortinget.no/eksport/publikasjoner?key=' + API_key + '&publikasjontype=referat&sesjonid=' #+ sesjonid
    publikasjoner_respons = http.request('GET', publikasjoner_url + sesjonsid)
    publikasjoner_root = et.fromstring(publikasjoner_respons.data)

    publikasjoner_liste = publikasjoner_root.find('{http://data.stortinget.no}publikasjoner_liste')

    for pub in publikasjoner_liste.findall('{http://data.stortinget.no}publikasjon'):
        if iterasjon <= nytt_format_ix:
            publikasjoner.append(pub.find('{http://data.stortinget.no}id').text)
        else:
            publikasjoner_eldreformat.append(pub.find('{http://data.stortinget.no}id').text)
    iterasjon += 1
print('\rFinished import of '  + str(len(sesjoner)) + ' reference-ids.',  end='')

In [945]:
# fetch person info to finn in party where missing
stortingsperioder = ['2017-2021', '2013-2017', '2009-2013', '2005-2009', '2001-2005', '1997-2001', '1993-97', '1989-93']
rep_list = []

iterasjon = 0
for periode in stortingsperioder:
    representanter_url = 'http://data.stortinget.no/eksport/representanter?key=' + API_key + '&stortingsperiodeid=' #+ periodeid
    representanter_respons = http.request('GET', representanter_url + periode)
    representanter_root = et.fromstring(representanter_respons.data)
    
    representanter_liste = representanter_root.find('{http://data.stortinget.no}representanter_liste')
    for rep in representanter_liste.findall('{http://data.stortinget.no}representant'):
        rep_id = rep.find('{http://data.stortinget.no}id').text
        rep_for = rep.find('{http://data.stortinget.no}fornavn').text
        rep_etter = rep.find('{http://data.stortinget.no}etternavn').text
        rep_parti = rep.find('{http://data.stortinget.no}parti')
        rep_partiid = rep_parti.find('{http://data.stortinget.no}id').text
        rep_partinavn = rep_parti.find('{http://data.stortinget.no}navn').text
        rep_row = [rep_id, rep_for + ' ' + rep_etter, rep_partiid, rep_partinavn, stortingsperioder[iterasjon]]
        rep_list.append(rep_row)
    iterasjon += 1

columns = ['id', 'navn', 'partiid', 'partinavn', 'stortingsperiode']
representanter_ses = pd.DataFrame(rep_list, columns=columns)

# add some members of government who have not been in parlament
representanter_legges_til = [
    ["VidHel", "Vidar Helgesen", "H", "Høyre", ""],
    ['MonMae', 'Monica Mæland', 'H', 'Høyre', ''],
    ['AnnHau', 'Anniken Hauglie', 'H', 'Høyre', ''],
    ['MarBer', 'Marit Berger Røsland', 'H', 'Høyre', ''],
    ['TerSoe', 'Terje Søviknes', 'FrP', 'Fremskrittspartiet', '']
]

# adds above list til dataframe
for li in representanter_legges_til:
    representanter_ses.loc[len(representanter_ses)] = li

# group by id to remove some duplicates
representanter = representanter_ses.groupby(['id']).first().reset_index().set_index('id')

In [976]:
# some functions for cleaning strings and fetching party from representanter-dataframe

funksjoner =["Statsminister", "Statsråd", "Utenriksminister", "Stortingspresident"]

def move_function_from_name(string):
    # subfunction of clean_representant_string()
    # take string, return list with name[0], function[1]
    navn = string
    funksjon = ''
    string_words = string.split()
    if (len(string_words) > 0) and (string_words[0] in funksjoner):
        navn = ' '.join(string_words[1:])
        funksjon = string_words[0]
    return [navn, funksjon]

def clean_representant_string(string):
    # take string found in newer http://data.stortinget.no/eksport/publikasjon?publikasjonid
    # return list with name [0] and party [1], elective_function[2]
    
    rep_navn = re.search(r"^[^\(|^\[]*", string)
    if rep_navn: 
        # lots of noise in xml files which complicated party lookup
        rep_navn = rep_navn.group(0).strip()
        rep_navn = rep_navn.replace('\n', ' ')
        rep_navn = rep_navn.replace('\xad', '')
        rep_navn = rep_navn.strip(':')
    else:
        rep_navn = "unknown"
    
    rep_navn, rep_funksjon = move_function_from_name(rep_navn)
    
    # fetch party from string in xml inside ()
    rep_parti = re.search(r"\(([A-Za-z0-9_]+)\)", string)
    if rep_parti: 
        rep_parti = rep_parti.group(1)
    else:
        # else fetch party from representanter-dataframe
        if not representanter.loc[representanter.navn == rep_navn.strip(), 'partiid'].empty:
            rep_parti = representanter.loc[representanter.navn == rep_navn.strip(), 'partiid'][0]
        else:
            rep_parti = "unknown"
    
    return([rep_navn, rep_parti, rep_funksjon])

In [ ]:
# by 2018-03-25 there is about 180 documents with new format (about 34 MB in sqlite database)
# and ~1622 with old format (this plus the above is about 268 MB in sqlite database)

# new foramt id-s stored in list: publikasjoner
# old foramt id-s stored in list: publikasjoner_eldreformat

# first fetch the new format docs, and store in sqlite database, table 'corpus'

#ref_liste = [] # for testing
ref_rad = []

referat_juks = [publikasjoner[75]]
iterasjon = 0
for referat in publikasjoner: # publikasjoner eller referat_juks for test
    print('\rImport referat ' + str(iterasjon + 1) + ' av ' + str(len(publikasjoner)),  end='')
    iterasjon += 1

    referat_url = 'http://data.stortinget.no/eksport/publikasjon?key=' + API_key + '&publikasjonid='
    referat_respons = http.request('GET', referat_url + referat) # referat
    referat_root = et.fromstring(referat_respons.data)
    referat_mote = referat_root.find('Mote')
    if referat_mote is not None:
        referat_hoved = referat_mote.find('Hovedseksjon')
    else: 
        print('\nSkipping ' + referat)
        continue
        
    if referat_hoved is not None:
        referat_saker = referat_hoved.find('Saker') # breaks on 35
    else:
        print('\nSkipping ' + referat)
        continue

    for sak in referat_saker.findall('*'):
        for hoved in sak.findall('Hovedinnlegg'):
            for all_as in hoved.findall('A'):
                navn = all_as.find('Navn')
                if navn is not None: 
                    if len(ref_rad) > 0: 
                        #ref_liste.append(ref_rad) # for test
                        cursor.execute('''INSERT INTO corpus(name, party, function, maintext) VALUES(?,?,?,?)''', (r_navn, r_parti, r_funksjon, ref_rad[-1]))
                    ref_rad = []
                    if navn.text is not None:
                        r_navn, r_parti, r_funksjon = clean_representant_string(navn.text)
                        ref_rad.append(r_navn)
                        ref_rad.append(r_parti)
                        ref_rad.append(r_funksjon)
                    if navn.tail is not None:
                        ref_rad.append(navn.tail.replace('\n', ' ').replace(u'\xa0', u' ').strip())
                    #print(ref_rad) # test stuff
                ref_rad[-1] = ref_rad[-1] + str(all_as.text).replace('\n', ' ').replace(u'\xa0', u' ').strip()
    cursor.execute('''INSERT INTO corpus(name, party, function, maintext) VALUES(?,?,?,?)''', (r_navn, r_parti, r_funksjon, ref_rad[-1]))
    db_con.commit()
            
#len(ref_liste)

In [ ]:
# now to fetch documents with old format, with id-s from 'publikasjoner_eldreformat'

ref_liste = []
ref_rad = []
parse_errors = []

referat_test = [publikasjoner_eldreformat[1205]]
iterasjon = 0
for referat in publikasjoner_eldreformat: # publikasjoner_eldreformat eller referat_test for test
    print('\rImport referat ' + str(iterasjon + 1) + ' av ' + str(len(publikasjoner_eldreformat)),  end='')
    iterasjon += 1

    referat_url = 'http://data.stortinget.no/eksport/publikasjon?key=' + API_key + '&publikasjonid='
    referat_respons = http.request('GET', referat_url + referat) # referat
    # some cheking of valid xml? last 27 in list not valid
    try:
        referat_root = et.fromstring(referat_respons.data) # legge til error handling pga format
    except:
        parse_errors.append(referat)
        continue
    
    referat_mote = referat_root.find('mote')

    if referat_mote is None:
        print('skipping' + referat)
        continue
    else:
        referat_innlegg = referat_mote.findall('./saker/sak/innlegg') #navn,a,a,a..
        referat_innlegg = referat_innlegg + referat_mote.findall('./saker/sakreferat/referat/innlegg') #navn,a,a,a..
    
    # en eller annen funksjon som høster navn og alle a-er

    for innlegg in referat_innlegg:
        # new innlegg
        r_innlegg = ''
        
        navn = innlegg.find('navn')
        if (navn is not None) and (navn.text is not None):
            r_navn, r_parti, r_funksjon = clean_representant_string(navn.text)
        for all_as in innlegg.findall('a'):
            # append all the a-s in variable: r_innlegg
            r_innlegg = r_innlegg + str(all_as.text).replace('\n', ' ').replace('                   ', ' ').replace(u'\xa0', u' ').strip()
        
        #print('\n' + r_navn, r_parti, r_funksjon + '\n' + r_innlegg) # test stuff
        #print(r_navn, r_parti) # test stuff
        #print(repr(r_navn)) # test stuff for those hidden/strange characters
        
        # add to db for each speech by person
        cursor.execute('''INSERT INTO corpus(name, party, function, maintext) VALUES(?,?,?,?)''', (r_navn, r_parti, r_funksjon, r_innlegg))
    
    # commit to db for each document
    db_con.commit()

# a lot of documents in old format couldnt be parsed, so print this number at the end
# dident try find workaroudn of these errors
print('\n' + str(len(parse_errors)) + ' errors parsing')

In [984]:
# write references which gave parse errors to a file 
thefile = open('parse_errors.txt', 'w')

for item in parse_errors:
  thefile.write("%s\n" % item)

thefile.close()

In [989]:
# for testing if i need to add more representatives
# a lot of these are from invited speakers, or nonspecific names
# but there are also a lot who are connected to a party
# this can definetely be improved upon
sql = "SELECT name FROM corpus WHERE party=?" #  limit 100
cursor.execute(sql, [("unknown")]) # now renamed this ukjent
print(cursor.fetchall())

In [990]:
db_con.close()